In [18]:
import pandas as pd

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

stay = pd.read_csv('../Data/Output/Stay/153.csv', index_col=0)
stay['stime'] = pd.to_datetime(stay['stime'])

# stay.head()

stay['stimestamp'] = stay['stime'].astype('int64') // 1e9
# stay.head()

s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

# import datetime
import numpy as np

i = 0

maxrow = 128

result = np.empty((0, 128, 3))
# print(result.shape)

for g in s1:
    key = g[0].month
    # print(type(key))
    # .to_list()
    value = g[1][['grid', 'stimestamp', 'duration']].values
    
    # value.dropna(axis=0, how='all', inplace=True)
    value = value[value.sum(axis=1)!=0,:]
    # 如果行数为0，也就是说没有轨迹点。那么就跳过。
    if value.shape[0] == 0:
        continue
    # 将轨迹填充为相同的形状。
    if value.shape[0] > maxrow:
        continue
    else:
        value = np.pad(array=value, pad_width=((0,maxrow-value.shape[0]),(0,0)), mode='constant')
    
    # print(value.shape)
    # print(result.shape)
    # d = np.concatenate((a[np.newaxis,:], b[np.newaxis,:]), axis=0)
    result = np.concatenate((result, value[np.newaxis,:]), axis=0)
    # if i < 2:
        
    #     print(value)
    #     i += 1
    # print(type(value))
    # output type is <class 'numpy.ndarray'>
    # np.set_printoptions(suppress=True)
    # print(value)

a2d = result.reshape(result.shape[0], -1)
# np.savetxt('{}.csv'.format(153), a2d, delimiter="," )
import csv
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(a2d)
print(result.shape)

(52, 128, 3)


In [21]:
import csv
import numpy as np

# 从csv文件读取2D数组
with open("output.csv", "r") as f:
    reader = csv.reader(f)
    a2d = np.array(list(reader)).astype("float")

# 将2D数组转换为3D数组
a = a2d.reshape(-1, 128, 3)
print(a.shape)

(52, 128, 3)


In [ ]:
import pandas as pd

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

def tte(user):
    stay = pd.read_csv('./Data/Output/Stay/{}.csv'.format(user), index_col=0)
    stay['stime'] = pd.to_datetime(stay['stime'])
    s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

    gridList = []
    for g in s1:
        key = g[0].month
        # print(type(key))
        value = g[1]['grid'].to_list()
        gridList.append(value)
        # print(key)
        # print(value)
    gridpd = pd.DataFrame(gridList)
    gridpd = drop_all_0_row(gridpd)
    gridpd.dropna(axis=0, how='all', inplace=True)
    print(gridpd.shape)
    return gridpd